<a href="https://colab.research.google.com/github/ArshHp/LLM/blob/main/kg_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Read the datafile from inputfile_extractor and process it Create the knowledge graph for the documents


In [52]:
!pip install -q load_dotenv

In [53]:
!pip install -q langchain_community

In [54]:
from dotenv import load_dotenv
import os

# Common data processing
import json
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain



# Warning control
import warnings
warnings.filterwarnings("ignore")

In [55]:
!pip install -q neo4j

NEO4J_URI = 'neo4j+s://ec3e319b.databases.neo4j.io:7687'

NEO4J_USERNAME = 'neo4j'

NEO4J_PASSWORD = 'o1supQkFCHQPMOG1Z9Usk39RU8Wf6FzAbPhDxEVhZEo'

NEO4J_DATABASE = 'neo4j

In [56]:
NEO4J_URI = 'neo4j+s://ec3e319b.databases.neo4j.io:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'o1supQkFCHQPMOG1Z9Usk39RU8Wf6FzAbPhDxEVhZEo'
NEO4J_DATABASE = 'neo4j'

In [57]:
# Load from environment
load_dotenv('.env', override=True)
#NEO4J_URI = os.getenv('NEO4J_URI')
#NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
#NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
#NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

# Global constants
VECTOR_INDEX_NAME = 'opsb_suite_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [58]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
Inputfile = 'drive/My Drive/output.json'

In [60]:
first_file_as_object = json.load(open(Inputfile))

In [61]:
import json

# Initialize an empty list to hold the parsed JSON objects
json_objects = []

# Iterate through each item in the list and convert it to a dictionary
for item in first_file_as_object:
    try:
        json_object = json.loads(item)  # Parse the string into a dictionary
        json_objects.append(json_object)  # Append to the list of JSON objects
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

# Convert the list of dictionaries into a JSON string
json_result = json.dumps(json_objects, indent=4)

# Print the final JSON object
#print(json_result)
with open('temp.json','w') as json_file:
    json.dump(json_objects,json_file,indent=4)
print ("JSON file created successfully")

JSON file created successfully


In [62]:
first_file_as_object_new = json.load(open('temp.json'))

In [63]:
for k,v in first_file_as_object_new[0].items():
    print(k, type(v))

Issue <class 'str'>
Cause <class 'str'>
Solution <class 'str'>


In [64]:
item1_issue = first_file_as_object_new[0]['Issue']
print (item1_issue)

Failed creating reader to topic in scheduler
pod log

When deploying AWS in a shared OPTIC Data Lake environment with Operations Bridge as provider, an error was encountered in the
scheduler pod NFS logs.


In [65]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 50,
    chunk_overlap  = 5,
    length_function = len,
    is_separator_regex = False,
)

In [66]:
item1_issues_chunks = text_splitter.split_text(item1_issue)

In [67]:
type(item1_issues_chunks)

list

In [68]:
len(item1_issues_chunks)

6

In [69]:
item1_issues_chunks[0]

'Failed creating reader to topic in scheduler'

In [70]:
def getrecords(first_file_as_object,content_source):
  chunks_with_metadata = [] # use this to accumlate chunk records
  chunk_seq_id = 0
  for item in first_file_as_object:
    for data in ['Issue','Cause','Solution']:# pull these keys from the json
      item_text = item[data] # grab the text of the item
      #item_text_chunks = text_splitter.split_text(item_text) # split the text into chunks
      item_text_chunks = item_text # Chunk is too small so not doing check
      chunks_with_metadata.append({
                'text': item_text_chunks,
                'contentType': data,
                # metadata from looping...
                'chunkSeqId': chunk_seq_id,
                'source':content_source
                # metadata from file...
                #Add the metadata for files like filename etc in future
            })
      chunk_seq_id += 1
    return chunks_with_metadata


In [71]:
response = getrecords(first_file_as_object_new,VECTOR_INDEX_NAME)
print ("Chunks for knowledge graph:\n", response)


Chunks for knowledge graph:
 [{'text': 'Failed creating reader to topic in scheduler\npod log\n\nWhen deploying AWS in a shared OPTIC Data Lake environment with Operations Bridge as provider, an error was encountered in the\nscheduler pod NFS logs.', 'contentType': 'Issue', 'chunkSeqId': 0, 'source': 'opsb_suite_chunks'}, {'text': 'This occurs when there is a connection error between the Message Bus and Vertica. Following is the error recorded in the scheduler pod\nNFS logs:\nFailed creating reader to topic - requesting restart: ConnectError | Zero message count found in topic!', 'contentType': 'Cause', 'chunkSeqId': 1, 'source': 'opsb_suite_chunks'}, {'text': 'Perform the following steps as a workaround to resolve this error:\n1. Run the following command to add the existing OpsB suite chart values to the helmvalues_opsb.yaml:\nhelm get values opsb -n opsb -o yaml > helmvalues_opsb.yaml\n2. Update the following parameters in the helmvalues_opsb.yaml file and set the values as shown be

In [73]:
merge_chunk_node_query = """
MERGE(mergedChunk:Chunk {chunkId: $chunkParam.chunkSeqId})
    ON CREATE SET
        mergedChunk.contentType = $chunkParam.contentType,
        mergedChunk.chunkSeqId = $chunkParam.chunkSeqId,
        mergedChunk.text = $chunkParam.text,
        mergedChunk.source = $chunkParam.source
RETURN mergedChunk
"""

In [51]:
NEO4J_URI = 'neo4j+s://ec3e319b.databases.neo4j.io:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'o1supQkFCHQPMOG1Z9Usk39RU8Wf6FzAbPhDxEVhZEo'
NEO4J_DATABASE = 'neo4j'

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [76]:
kg.query(merge_chunk_node_query,
         params={'chunkParam':response[0]})

[{'mergedChunk': {'text': 'Failed creating reader to topic in scheduler\npod log\n\nWhen deploying AWS in a shared OPTIC Data Lake environment with Operations Bridge as provider, an error was encountered in the\nscheduler pod NFS logs.',
   'source': 'opsb_suite_chunks',
   'chunkId': 0,
   'contentType': 'Issue',
   'chunkSeqId': 0}}]

In [77]:
kg.query("""
CREATE CONSTRAINT unique_chunk IF NOT EXISTS
    FOR (c:Chunk) REQUIRE c.chunkId IS UNIQUE
""")

[]

In [78]:
kg.query("SHOW INDEXES")

[{'id': 2,
  'name': 'Property_1_DocumentID_uniq',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'RANGE',
  'entityType': 'NODE',
  'labelsOrTypes': ['DocumentID'],
  'properties': ['Property 1'],
  'indexProvider': 'range-1.0',
  'owningConstraint': 'Property_1_DocumentID_uniq',
  'lastRead': neo4j.time.DateTime(2024, 9, 12, 14, 44, 42, 350000000, tzinfo=<UTC>),
  'readCount': 91},
 {'id': 0,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 9, 17, 15, 40, 22, 456000000, tzinfo=<UTC>),
  'readCount': 162},
 {'id': 1,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstrai

In [79]:
node_count = 0
for chunk in response:
    print(f"Creating `:Chunk` node for chunk ID {chunk['chunkSeqId']}")
    kg.query(merge_chunk_node_query,
            params={
                'chunkParam': chunk
            })
    node_count += 1
print(f"Created {node_count} nodes")

Creating `:Chunk` node for chunk ID 0
Creating `:Chunk` node for chunk ID 1
Creating `:Chunk` node for chunk ID 2
Created 3 nodes


In [80]:
kg.query("""
         MATCH (n)
         RETURN count(n) as nodeCount
         """)

[{'nodeCount': 134}]

In [95]:
kg.query("""
    DROP INDEX `opsb_suite_chunks` IF EXISTS
""")

[]

In [96]:
kg.query("""
         CREATE VECTOR INDEX `opsb_suite_chunks` IF NOT EXISTS
          FOR (c:Chunk) ON (c.textEmbedding)
          OPTIONS { indexConfig: {
            `vector.dimensions`: 768,
            `vector.similarity_function`: 'cosine'
         }}
""")

[]

In [97]:
kg.query("SHOW INDEXES")

[{'id': 2,
  'name': 'Property_1_DocumentID_uniq',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'RANGE',
  'entityType': 'NODE',
  'labelsOrTypes': ['DocumentID'],
  'properties': ['Property 1'],
  'indexProvider': 'range-1.0',
  'owningConstraint': 'Property_1_DocumentID_uniq',
  'lastRead': neo4j.time.DateTime(2024, 9, 12, 14, 44, 42, 350000000, tzinfo=<UTC>),
  'readCount': 91},
 {'id': 0,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 9, 17, 16, 44, 11, 797000000, tzinfo=<UTC>),
  'readCount': 360},
 {'id': 1,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstrai

In [84]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load the tokenizer and model for E5-base
model_name = "intfloat/e5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Define a function to get the embeddings
def get_text_embedding(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Pass the tokenized input through the model to get the hidden states
    with torch.no_grad():
        outputs = model(**inputs)

    # Take the output embeddings of the [CLS] token, or you can use pooling for sentence embeddings
    embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token is at index 0
    return embeddings.squeeze().numpy()  # Return as numpy array for easier manipulation

# Sample text
text = "This is a sample text for embedding extraction."

# Get the embedding for the text
embedding = get_text_embedding(text)

print("Embedding shape:", embedding.shape)
print("Embedding vector:", embedding)


tokenizer_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Embedding shape: (768,)
Embedding vector: [-2.23132148e-01  4.36401159e-01 -3.79955381e-01 -4.49896246e-01
  5.83398044e-01  4.76735592e-01  1.13116808e-01  2.92197287e-01
 -7.99743891e-01  1.30592152e-01 -5.95337093e-01  3.64148542e-02
 -3.96158934e-01 -7.98064172e-01 -2.97484070e-01 -3.07159901e-01
  3.57971668e-01 -2.36148015e-01  2.53480077e-01 -4.21074539e-01
 -3.60839278e-01 -4.81658340e-01  8.13271999e-01 -2.21854523e-01
 -5.51370502e-01  7.92325377e-01  4.14675713e-01  4.92017299e-01
 -6.90166235e-01 -4.37615812e-01  3.20250511e-01  1.96448803e-01
 -3.86156797e-01 -4.61367011e-01  4.70961407e-02  3.04419607e-01
 -9.48509812e-01 -2.81348407e-01 -1.02630578e-01  1.04570217e-01
 -2.73404866e-01  9.56359208e-01  1.41345650e-01 -3.40980977e-01
 -4.82236534e-01 -5.15102088e-01 -9.89346683e-01  1.12567198e+00
 -8.36809695e-01 -9.80176091e-01 -1.14665353e+00  1.58601031e-01
  1.05950129e+00 -1.47914499e-01 -7.12537169e-01  5.38705170e-01
  9.94031131e-02 -5.24931610e-01  3.15915257e-01

In [98]:
# Usage
# Assuming `kg` is your knowledge graph instance
#update_kg_with_e5_embeddings(kg)

# Function to query the knowledge graph and update with E5-base embeddings
def update_kg_with_e5_embeddings(kg):
    # Query the knowledge graph and find chunks with missing text embeddings
    result = kg.query("""
        MATCH (chunk:Chunk) WHERE chunk.textEmbedding IS NULL
        RETURN chunk.text AS text
    """)
    # Iterate over the results and update the embeddings
    for record in result:
        text = record["text"]
        embedding = get_text_embedding(text)
        kg.query("""
            MATCH (chunk:Chunk {text: $text})
            CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", $embedding)
            RETURN chunk
        """, params={"text": text, "embedding": embedding})

In [99]:
update_kg_with_e5_embeddings(kg)

In [100]:
kg.refresh_schema()
print(kg.schema)

Node properties:
DocumentID {Property 1: STRING}
Person {id: STRING, name: STRING, human_readable_id: INTEGER, text_unit_ids: LIST, description: STRING, type: STRING, description_embedding: LIST}
ACCOUNT {id: STRING, name: STRING, human_readable_id: INTEGER, text_unit_ids: LIST, description: STRING, type: STRING, description_embedding: LIST}
ACTION {id: STRING, name: STRING, human_readable_id: INTEGER, text_unit_ids: LIST, description: STRING, type: STRING, description_embedding: LIST}
ACTIVITY {id: STRING, name: STRING, human_readable_id: INTEGER, text_unit_ids: LIST, description: STRING, type: STRING, description_embedding: LIST}
APPLICATION {id: STRING, name: STRING, human_readable_id: INTEGER, text_unit_ids: LIST, description: STRING, type: STRING, description_embedding: LIST}
CALENDAR {id: STRING, name: STRING, human_readable_id: INTEGER, text_unit_ids: LIST, description: STRING, type: STRING, description_embedding: LIST}
CONCEPT {id: STRING, name: STRING, human_readable_id: INTEG

In [101]:
def neo4j_vector_search(question):
  question_embedding = get_text_embedding(question)
  """Search for similar nodes using the Neo4j vector index"""
  vector_search_query = """
    WITH $question_embedding AS question_embedding
    CALL db.index.vector.queryNodes($index_name, $top_k, question_embedding) yield node, score
    RETURN score, node.text AS text
  """
  similar = kg.query(vector_search_query,
                     params={
                      'index_name':VECTOR_INDEX_NAME,
                      'top_k': 10,
                      'question_embedding': question_embedding
                      })
  return similar

In [104]:
search_results = neo4j_vector_search(
    'Failed creating scheduler.'
)

In [105]:
search_results[0]

{'score': 0.8661801815032959,
 'text': 'Failed creating reader to topic in scheduler\npod log\n\nWhen deploying AWS in a shared OPTIC Data Lake environment with Operations Bridge as provider, an error was encountered in the\nscheduler pod NFS logs.'}